In [1]:
!pip install transformers torch datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 90.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 56.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 88.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 485.4/485.4 kB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

# Model name
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Load tokenizer and model with authentication
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name).to("cuda" if torch.cuda.is_available() else "cpu")

# Define the text-to-sql generation pipeline
pre_pipe = pipeline("text-to-sql-generation", model=model, tokenizer=tokenizer, device=0 if torch.cuda.is_available() else -1)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:823: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/3.07k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

Device set to use cuda:0


In [8]:
# Pre-Fine Tuning

# System instruction for formatting SQL and explanation
instruction = (
    "Generate the natural language query and schema into SQL. "
    "Return the SQL code in a separate code block, formatted correctly, followed by an explanation."
)

# User query with schema and request
prompt_user = """
What is the maximum number of genetic research projects in Japan per year?

schema: CREATE SCHEMA if not exists genetic;CREATE TABLE if not exists genetic.projects
(id INT PRIMARY KEY, name VARCHAR(100), start_date DATE);INSERT INTO genetic.projects
(id, name, start_date) VALUES (1, 'ProjectX', '2018-01-01'), (2, 'ProjectY', '2020-05-15'),
(3, 'ProjectZ', '2017-08-08'), (4, 'ProjectW', '2021-03-03'), (5, 'ProjectV', '2020-09-20'), (6, 'ProjectU', '2019-12-31');


"""

# Construct prompt
prompt = f"{instruction}\n\nUser Query:\n{prompt_user}"

# Generate SQL output
with torch.no_grad():  # Disable gradient tracking to save memory
    response = pre_pipe(prompt, max_length=512, do_sample=True, temperature=0.7)

# Print the generated SQL query and explanation
print(response[0]['generated_text'])


Generate the natural language query into SQL. Return the SQL code in a separate code block, formatted correctly, followed by an explanation.

User Query:

What is the maximum number of genetic research projects in Japan per year?

The natural language query is: "What is the maximum number of genetic research projects in Japan per year?"

To get the SQL code, we'll need to query the database table that contains this information.

Assuming the table is called 'genetic_projects' and the column that contains the data is'max_projects', which is of type INTEGER.

We can use the LIMIT clause with 1 to fetch the maximum value.

So the SQL query would be: SELECT MAX(max_projects) AS max_projects FROM genetic_projects LIMIT 1;

But let's verify the table structure.

Looking at the table, there are multiple projects, each with details like project_id, country, dates, etc. The'max_projects' column holds the number of projects in each country.

Therefore, the code above should correctly return the 

In [9]:
from datasets import load_dataset
import pandas as pd

# Load the dataset
dataset = load_dataset("gretelai/synthetic_text_to_sql", split='train')

# Filter rows where 'domain' is 'wildlife conservation'
filtered_dataset = dataset.filter(lambda x: x['domain'] == 'biotechnology')

# Convert filtered dataset to a pandas DataFrame for easier manipulation
df = filtered_dataset.to_pandas()

# Remove rows with any null values
df_filtered = df.dropna()

# Convert back to a Hugging Face Dataset for further processing
clean_dataset = dataset.from_pandas(df_filtered)

# Check the number of rows after cleaning
print(f"Number of rows after removing nulls: {len(df_filtered)}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/8.18k [00:00<?, ?B/s]

(…)nthetic_text_to_sql_train.snappy.parquet:   0%|          | 0.00/32.4M [00:00<?, ?B/s]

(…)ynthetic_text_to_sql_test.snappy.parquet:   0%|          | 0.00/1.90M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/100000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/5851 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100000 [00:00<?, ? examples/s]

Number of rows after removing nulls: 947


In [10]:
# Set the pad_token to the eos_token
tokenizer.pad_token = tokenizer.eos_token

def tokenize_function(examples):
    # Tokenize the examples and shift the labels by one for causal language modeling
    inputs = tokenizer(examples['sql_prompt'], truncation=True, padding="max_length", max_length=128)
    labels = tokenizer(examples['sql'], truncation=True, padding="max_length", max_length=128)

    # Shift labels to the right by one position
    inputs['labels'] = labels['input_ids']

    return inputs

# Tokenize dataset and add labels
tokenized_dataset = clean_dataset.map(tokenize_function, batched=True)


print(tokenized_dataset[0])
print(tokenized_dataset['input_ids'])
print(tokenized_dataset['labels'])


Map:   0%|          | 0/947 [00:00<?, ? examples/s]

{'id': 5115, 'domain': 'biotechnology', 'domain_description': 'Genetic research data, bioprocess engineering information, biotech startup funding, and biosensor technology development.', 'sql_complexity': 'single join', 'sql_complexity_description': 'only one join (specify inner, outer, cross)', 'sql_task_type': 'analytics and reporting', 'sql_task_type_description': 'generating reports, dashboards, and analytical insights', 'sql_prompt': 'List startups that received funding for biosensor development', 'sql_context': "CREATE TABLE startups (id INT, name VARCHAR(50), domain VARCHAR(50), location VARCHAR(50)); INSERT INTO startups (id, name, domain, location) VALUES (1, 'SensBiotech', 'Biosensors', 'CA'); INSERT INTO startups (id, name, domain, location) VALUES (2, 'BioSignal', 'Bioinformatics', 'NY'); CREATE TABLE funding (id INT, startup_id INT, amount DECIMAL(10, 2), purpose VARCHAR(50)); INSERT INTO funding (id, startup_id, amount, purpose) VALUES (1, 1, 50000.00, 'Biosensor Developm

In [11]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model
import torch

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Define LoRA configuration
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM",
)

# Apply LoRA to the model
model = get_peft_model(model, lora_config)

# Define training arguments
training_args = TrainingArguments(
    output_dir="./finetuned_model",
    evaluation_strategy="no",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    num_train_epochs=3,
    save_steps=500,
    fp16=True,
    save_total_limit=2,
    push_to_hub=False,
    report_to="none"

)

# Define Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    tokenizer=tokenizer,
)

tokenizer.pad_token = tokenizer.eos_token

# Train the model
trainer.train()

# Save LoRA adapter
model.save_pretrained("./finetuned_model")
tokenizer.save_pretrained("./finetuned_model")



/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-11-16c2f0b4a2f5>:37: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


Step,Training Loss
500,4.129800


('./fine_tuned_lora/tokenizer_config.json',
 './fine_tuned_lora/special_tokens_map.json',
 './fine_tuned_lora/tokenizer.json')

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

# Define your base model name
base_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

# Load the base model
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name, torch_dtype=torch.float16, device_map="auto"
)

# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_name)

# Now load the fine-tuned LoRA adapter
fine_tuned_model = PeftModel.from_pretrained(base_model, "./finetuned_model")

# Merge LoRA adapter into base model for inference
fine_tuned_model = fine_tuned_model.merge_and_unload()

# Move the model to GPU (if available)
device = "cuda" if torch.cuda.is_available() else "cpu"
fine_tuned_model.to(device)


# Define the text-generation pipeline for the fine-tuned model
post_fine_tuned = pipeline("text-to-sql-generation", model=fine_tuned_model, tokenizer=tokenizer if torch.cuda.is_available() else -1)

# System instruction for formatting SQL and explanation
instruction = (
    "Generate the natural language query and schema into SQL. "
    "Return the SQL code in a separate code block, formatted correctly, followed by an explanation."
)

# User query with schema and request
prompt_user = """
What is the maximum number of genetic research projects in Japan per year?

schema: CREATE SCHEMA if not exists genetic;CREATE TABLE if not exists genetic.projects
(id INT PRIMARY KEY, name VARCHAR(100), start_date DATE);INSERT INTO genetic.projects
(id, name, start_date) VALUES (1, 'ProjectX', '2018-01-01'), (2, 'ProjectY', '2020-05-15'),
(3, 'ProjectZ', '2017-08-08'), (4, 'ProjectW', '2021-03-03'), (5, 'ProjectV', '2020-09-20'), (6, 'ProjectU', '2019-12-31');


"""

# Construct prompt
prompt = f"{instruction}\n\nUser Query:\n{prompt_user}"

# Generate SQL output from the fine-tuned model
with torch.no_grad():  # Disable gradient tracking to save memory
    response = post_fine_tuned(prompt,
                           max_length=512,
                           do_sample=True,
                           temperature=0.7,
                           repetition_penalty=1.2,
                           top_k=50,
                           top_p=0.9)

# Print the generated SQL query and explanation from the fine-tuned model
print(response[0]['generated_text'])
# Tokenize the response
tokenized_response = tokenizer(response[0]['generated_text'], return_tensors='pt', )


Device set to use cuda:0
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


Translate the natural language query into SQL. Return the SQL code in a separate code block, formatted correctly, followed by an explanation.

User Query:

What is the maximum number of genetic research projects in Japan per year?

The user wants to know how many genetic research projects are conducted annually in Japan.
</think>

To determine the maximum number of genetic research projects in Japan per year using SQL, we can write the following query:

```sql
SELECT MAX(genetic_research_projects) AS max genetically_research_projects FROM your_table;
```

This will return the highest value from the `genetic_research_projects` column for all records in your table named "your_table".

### Explanation:
- **SELECT MAX(...):** This selects the maximum value of the specified column (`genetic_research_projects`). 
- **AS max genetically_research_projects:** Renames the result set field to make it clear and descriptive.
- **FROM your_table:**
  - The term 'your_table' should be replaced with t

In [14]:
!pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.1/322.1 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 5.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [15]:
import gradio as gr

# Function to generate SQL from natural language query
def generate_sql_response(nl_query, db_schema):
    instruction = (
        "Generate the natural language query and schema into SQL. "
        "Ensure the SQL is properly formatted and include a brief explanation."
    )
    prompt = f"{instruction}\n\nDatabase Schema:\n{db_schema}\n\nUser Prompt:\n{nl_query}"

    with torch.no_grad():
        response = post_fine_tuned(prompt,
                                   max_length=1024,
                                   do_sample=True,
                                   temperature=0.7,
                                   repetition_penalty=1.2,
                                   top_k=50,
                                   top_p=0.9)

    return response[0]['generated_text']

# Define the Gradio interface
with gr.Blocks() as sql_generator:
    gr.Markdown("## SQL Query Generator 🚀")
    gr.Markdown("Convert natural language queries into SQL queries with explanations.")

    with gr.Row():
        with gr.Column():
            query_box = gr.Textbox(
                label="Enter Natural Language Query",
                placeholder="Example: Show all employees who joined after 2020",
                lines=3
            )

            schema_box = gr.Textbox(
                label="Provide Database Schema",
                placeholder="Tables: employees(id, name, department, join_date), salaries(emp_id, salary, bonus)",
                lines=5
            )

            generate_button = gr.Button("Generate SQL Query")

        with gr.Column():
            sql_output = gr.Markdown(label="Generated SQL Query and Explanation")

    generate_button.click(
        fn=generate_sql_response,
        inputs=[query_box, schema_box],
        outputs=sql_output
    )

# Launch Gradio app
sql_generator.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://3b3144a7d8f0527a49.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
